In [1]:
import requests
import pandas as pd

BASE_URL = "https://www.cheapshark.com/api/1.0/deals"

def fetch_all_deals(store_id="1", page_size=60, max_pages=30):
    """
    Fetch multiple pages of CheapShark deals.

    - store_id="1" means Steam
    - page_size=60 is the maximum number of deals per page
    - max_pages is a safety limit so we don't loop forever
    """
    all_deals = []
    page = 0

    while page < max_pages:
        params = {
            "storeID": store_id,
            "pageSize": page_size,
            "pageNumber": page,
            "sortBy": "Savings",  # sort by biggest discount first
        }
        print(f"Requesting page {page} ...")
        resp = requests.get(BASE_URL, params=params, timeout=30)

        if resp.status_code != 200:
            print(f"  ❌ Status code {resp.status_code}, stopping.")
            break

        deals = resp.json()
        print(f"  Got {len(deals)} deals.")

        if not deals:
            # No more data, we've reached the end
            break

        all_deals.extend(deals)

        # If we got fewer than page_size deals, this is probably the last page
        if len(deals) < page_size:
            break

        page += 1

    print(f"\nTotal deals fetched: {len(all_deals)}")

    df = pd.DataFrame(all_deals)

    # Convert CheapShark's releaseDate (Unix seconds) to datetime, year, and a readable date string
    if "releaseDate" in df.columns:
        df["release_date"] = pd.to_datetime(
            df["releaseDate"],
            unit="s",
            utc=True,
            errors="coerce"
        )
        df["release_year"] = df["release_date"].dt.year

        # 🔹 New column: formatted date string (e.g. 01/01/70, 10/04/16, etc.)
        df["date"] = df["release_date"].dt.strftime("%m/%d/%y")

    return df


df_raw = fetch_all_deals()
df_raw.head()


Requesting page 0 ...


  Got 60 deals.
Requesting page 1 ...


  Got 60 deals.
Requesting page 2 ...


  Got 60 deals.
Requesting page 3 ...


  Got 60 deals.
Requesting page 4 ...


...

  Got 60 deals.
Requesting page 27 ...


  Got 60 deals.
Requesting page 28 ...


  Got 60 deals.
Requesting page 29 ...


  Got 60 deals.

Total deals fetched: 1800


,internalName,title,metacriticLink,dealID,storeID,gameID,salePrice,normalPrice,isOnSale,savings,...,steamRatingPercent,steamRatingCount,steamAppID,releaseDate,lastChange,dealRating,thumb,release_date,release_year,date
0,BAKERYSIMULATOR,Bakery Simulator,/game/bakery-simulator/,CgQ3bEtWuNSfva1MNESmK0voFRJXFy4TRQv1KMlKZA8%3D,1,245267,0.79,15.99,1,95.059412,...,66,1937,1031120,1651536000,1766088015,8.9,https://shared.fastly.steamstatic.com/store_it...,2022-05-03 00:00:00+00:00,2022,05/03/22
1,EXTREMEOVERTAKE,Extreme Overtake,/game/extreme-overtake/,hRNgfi3%2FRW4QkF9fNTcDSbIl39E8BidRAmmO9Unc%2Bc...,1,304677,0.79,15.99,1,95.059412,...,0,0,3138100,1733529600,1765568791,4.0,https://shared.fastly.steamstatic.com/store_it...,2024-12-07 00:00:00+00:00,2024,12/07/24
2,JUSTMOVEFALLDUNGEONENDLESSABYSS,Just Move Fall Dungeon Endless Abyss,None,0n%2FnB%2By%2Bk3G785meL6tDZ%2FwDfvF2ee2zFJEsxX...,1,306530,0.79,15.99,1,95.059412,...,0,0,3856040,0,1764884537,2.0,https://shared.fastly.steamstatic.com/store_it...,1970-01-01 00:00:00+00:00,1970,01/01/70
3,LIVINGBATTLE,LivingBattle,None,6%2FJawAxHTcqG3WqvYHKvOPkqn116HLHEZljlb4EcV8k%3D,1,312088,0.79,15.99,1,95.059412,...,0,0,3353830,0,1765216975,2.6,https://shared.fastly.steamstatic.com/store_it...,1970-01-01 00:00:00+00:00,1970,01/01/70
4,UNCLAIMEDWORLD,Unclaimed World,/game/unclaimed-world/,oOIeSWUAU6bwhCt8eVnsnELwb%2FXff3yYYMvmf%2FezlB...,1,107953,0.84,16.99,1,95.055915,...,49,174,284100,1475539200,1765233773,6.6,https://shared.fastly.steamstatic.com/store_it...,2016-10-04 00:00:00+00:00,2016,10/04/16


In [2]:
df_raw.to_csv("cheapshark_updated.csv", index=False)


In [3]:
# Create a smaller table for the range/arrow plot: top 20 deals by dealRating
df_top20 = (
    df_raw.copy()
    .assign(dealRating_float=pd.to_numeric(df_raw["dealRating"], errors="coerce"))
    .sort_values("dealRating_float", ascending=False)
    .head(20)
)

# Keep only the columns needed for Datawrapper
df_top20_for_chart = df_top20[["title", "normalPrice", "salePrice", "release_year"]]

df_top20_for_chart.head()


,title,normalPrice,salePrice,release_year
1761,Portal 2,9.99,1.99,2011
560,INSIDE,19.99,1.99,2016
767,The Witcher 3: Wild Hunt,39.99,3.99,2015
256,LIMBO,9.99,0.99,2011
250,Hitman 2: Silent Assassin,8.99,0.89,2002


In [4]:
df_top20_for_chart.to_csv("steam_deals_top20_rangeplot.csv", index=False)
